<a href="https://www.kaggle.com/code/rishabh0517/ubcbaselinetrain?scriptVersionId=157762093" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
from tqdm import tqdm
import os
import cv2
import numpy as np
from PIL import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import tensorflow as tf
import keras
from tensorflow.keras.applications import EfficientNetB0, InceptionV3
from tensorflow.keras.layers import Flatten,Dense,MaxPooling2D
from tensorflow.keras.models import Sequential
from keras.layers import Input, Dense,Conv2D , MaxPooling2D, Flatten,BatchNormalization,Dropout
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report,confusion_matrix
from collections import Counter
from tensorflow.keras.callbacks import ModelCheckpoint
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp
import torchvision
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import shuffle
from collections import Counter
from keras.preprocessing.image import ImageDataGenerator
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2
from sklearn.model_selection import StratifiedKFold

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
df=pd.read_csv('/kaggle/input/UBC-OCEAN/train.csv')

In [4]:
train_df_tma = df[df['is_tma']==True]

In [5]:
train_df_no_tma = df[df['is_tma']==False]

In [6]:
train_df_no_tma['image_id_path'] = [f"{i}_thumbnail.png" for i in train_df_no_tma['image_id']]

/tmp/ipykernel_42/2494151524.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_no_tma['image_id_path'] = [f"{i}_thumbnail.png" for i in train_df_no_tma['image_id']]


In [7]:
train_df_tma['image_id_path'] = [f"{i}.png" for i in train_df_tma['image_id']]

/tmp/ipykernel_42/1031206012.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df_tma['image_id_path'] = [f"{i}.png" for i in train_df_tma['image_id']]


In [8]:
image_data = []
image_label = []
img_size=150

In [9]:
for img , label in zip(train_df_no_tma['image_id_path'],train_df_no_tma['label']):
    image = Image.open("/kaggle/input/UBC-OCEAN/train_thumbnails/"+img)
    image = image.resize((img_size,img_size))
    image = image.convert("RGB")
    image = np.array(image)
    image_data.append(image)
    image_label.append(label)

In [10]:
for img , label in zip(train_df_tma['image_id_path'],train_df_tma['label']):
    image = Image.open("/kaggle/input/UBC-OCEAN/train_images/"+img)
    image = image.resize((img_size,img_size))
    image = image.convert("RGB")
    image = np.array(image)
    image_data.append(image)
    image_label.append(label)

In [11]:
datagen = ImageDataGenerator(
        rotation_range=20, # randomly rotate images by 20 degrees
        width_shift_range=0.2, # randomly shift images horizontally by 20% of the width
        height_shift_range=0.2, # randomly shift images vertically by 20% of the height
        horizontal_flip=True, # randomly flip images horizontally
        zoom_range=0.2)

In [12]:
def augment_and_balance_data(image_data, image_label, target_count):
    datagen = ImageDataGenerator(
        rotation_range=20,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        zoom_range=0.2
    )

    # Count the occurrences of each label
    label_counts = Counter(image_label)

    # Iterate through each label
    for label, count in label_counts.items():
        # Calculate the number of augmented images needed for this label
        num_augmented_images = target_count - count

        # If more images are needed for this label
        if num_augmented_images > 0:
            # Find indices of images with the current label
            indices = [i for i, l in enumerate(image_label) if l == label]

            # Randomly select existing images with this label
            selected_indices = np.random.choice(indices, num_augmented_images, replace=True)

            # Augment and append the new images and labels
            for i in selected_indices:
                img = image_data[i]
                img = img.reshape((1,) + img.shape)  # Add batch dimension
                label = image_label[i]

                for batch in datagen.flow(img, batch_size=1):
                    augmented_image = batch[0]
                    image_data.append(augmented_image)
                    image_label.append(label)
                    break  # Break to avoid infinite loop

# Example usage

In [13]:
# Assuming you have already loaded image_data and image_label
target_count = max(Counter(image_label).values())
print(target_count)# Set target count to the maximum label frequency
augment_and_balance_data(image_data, image_label, target_count)

222


In [14]:
print(len(image_data))
print(image_label.count('MC'))

1110
222


In [15]:
from sklearn.preprocessing import LabelEncoder

# Assuming image_label is your list of string labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(image_label)

In [17]:
image_data = [np.array(image) for image in image_data]

# Convert image_data and image_label to NumPy arrays
X = np.array(image_data)
Y = np.array(encoded_labels)

In [48]:
X.shape

(1110, 224, 224, 3)

In [18]:
datagen.fit(X)

# generate more images using the fitted generator
augmented_images_a = []
augmented_labels_a = []
for x_batch, y_batch in datagen.flow(X, Y, batch_size=len(X)):
    augmented_images_a.extend(x_batch)
    augmented_labels_a.extend(y_batch)
    break
for x_batch, y_batch in datagen.flow(X, Y, batch_size=len(X)):
    augmented_images_a.extend(x_batch)
    augmented_labels_a.extend(y_batch)
    break
for x_batch, y_batch in datagen.flow(X, Y, batch_size=len(X)):
    augmented_images_a.extend(x_batch)
    augmented_labels_a.extend(y_batch)
    break
for x_batch, y_batch in datagen.flow(X, Y, batch_size=len(X)):
    augmented_images_a.extend(x_batch)
    augmented_labels_a.extend(y_batch)
    break

In [19]:
augmented_images_a = np.array(augmented_images_a)
augmented_labels_a = np.array(augmented_labels_a)

#concatenate og data with augmented
X_train = np.concatenate((X, augmented_images_a))
y_train = np.concatenate((Y, augmented_labels_a))

In [20]:
X_train, y_train = shuffle(X_train,y_train, random_state=101)
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.3,random_state=101)

In [21]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(3885, 150, 150, 3)
(3885,)
(1665, 150, 150, 3)
(1665,)


In [22]:
y_train = tf.keras.utils.to_categorical(y_train)

y_test = tf.keras.utils.to_categorical(y_test)

In [54]:
class GeM(tf.keras.layers.Layer):
    def __init__(self, p=3, eps=1e-6):
        super(GeM, self).__init__()
        self.p = tf.constant(p, dtype=tf.float32)
        self.eps = eps

    def call(self, x):
        return self.gem(x, p=self.p, eps=self.eps)
        
    def gem(self, x, p=3, eps=1e-6):
        return tf.pow(tf.reduce_mean(tf.pow(tf.maximum(x, eps), p), axis=[1, 2]), 1.0/p)

In [72]:
import tensorflow_hub as hub
from tensorflow.keras.layers import Lambda
path1 = '/kaggle/input/keras-applications-models/EfficientNetB0.h5'
effnet = EfficientNetB0(weights=None, include_top=False, input_shape=(224,224, 3))
effnet.load_weights(path1)
# Define the Sequential model
num_class = 5
efficientnet_model = Sequential()

# Add layers to the Sequential model
efficientnet_model.add(effnet)
efficientnet_model.add(Dense(units=4096, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=4096, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=4096, activation="relu"))
efficientnet_model.add(tf.keras.layers.GlobalAveragePooling2D())
#efficientnet_model.add(Lambda(lambda x: GeM()(x)))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=2048, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=2048, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=1024, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=256, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=num_class, activation="softmax"))

In [26]:
path1 = '/kaggle/input/keras-applications-models/EfficientNetB0.h5'
effnet = EfficientNetB0(weights=None, include_top=False, input_shape=(150, 150, 3))
effnet.load_weights(path1)
# Define the Sequential model
num_class = 5
efficientnet_model = Sequential()

# Add layers to the Sequential model
efficientnet_model.add(effnet)
efficientnet_model.add(Dense(units=1024, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(tf.keras.layers.GlobalAveragePooling2D())
#efficientnet_model.add(Lambda(lambda x: GeM()(x)))
efficientnet_model.add(Dense(units=256, activation="relu"))
efficientnet_model.add(Dense(units=256, activation="relu"))
efficientnet_model.add(Dense(units=256, activation="relu"))
efficientnet_model.add(Dropout(0.25))
efficientnet_model.add(Dense(units=64, activation="relu"))
efficientnet_model.add(Dense(units=num_class, activation="softmax"))

In [27]:
opt = keras.optimizers.Adam(learning_rate=0.0001)
efficientnet_model.compile(loss='categorical_crossentropy',  optimizer=opt, metrics=['accuracy'])

In [28]:
class SaveBestWeights(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get('val_accuracy')
        val_loss = logs.get('val_loss')

        # Check if validation accuracy and recall are not None
        if val_accuracy is not None and val_loss is not None:
            filename = f'best_weights_epoch_{epoch:02d}_val_acc_{val_accuracy:.4f}_val_loss_{val_loss:.4f}.bin'
            self.model.save_weights(filename, save_format='tf')
            print(f"Saved best weights to {filename}")
        else:
            print(f"Warning: val_accuracy={val_accuracy}, val_loss={val_loss}. Skipping save.")

# Define a ModelCheckpoint callback to save the best weights based on validation accuracy
model_checkpoint = ModelCheckpoint('tmp_checkpoint.h5', save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)
save_best_weights_callback = SaveBestWeights()

# Train the model with the callbacks
history = efficientnet_model.fit(X_train, y_train, validation_split=0.3, epochs=30, verbose=1, batch_size=10)

Epoch 1/30


2024-01-03 21:00:31.187022: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential_1/efficientnetb0/block2b_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


272/272 [==============================] - 65s 67ms/step - loss: 1.3642 - accuracy: 0.4138 - val_loss: 1.0255 - val_accuracy: 0.5995
Epoch 2/30
272/272 [==============================] - 15s 54ms/step - loss: 0.8611 - accuracy: 0.6587 - val_loss: 0.5129 - val_accuracy: 0.8156
Epoch 3/30
272/272 [==============================] - 15s 54ms/step - loss: 0.5325 - accuracy: 0.7951 - val_loss: 0.3906 - val_accuracy: 0.8585
Epoch 4/30
272/272 [==============================] - 15s 54ms/step - loss: 0.3387 - accuracy: 0.8764 - val_loss: 0.3188 - val_accuracy: 0.8919
Epoch 5/30
272/272 [==============================] - 15s 54ms/step - loss: 0.2828 - accuracy: 0.9014 - val_loss: 0.3033 - val_accuracy: 0.9082
Epoch 6/30
272/272 [==============================] - 15s 54ms/step - loss: 0.2171 - accuracy: 0.9287 - val_loss: 0.2846 - val_accuracy: 0.9082
Epoch 7/30
272/272 [==============================] - 15s 54ms/step - loss: 0.1566 - accuracy: 0.9470 - val_loss: 0.2435 - val_accuracy: 0.9211
Epo

In [29]:
pred=np.argmax(efficientnet_model.predict(X_test),axis=1)
ytest=np.argmax(y_test,axis=1)
print(classification_report(pred,ytest))

53/53 [==============================] - 4s 31ms/step
              precision    recall  f1-score   support

           0       0.97      0.97      0.97       323
           1       0.96      0.92      0.94       342
           2       0.90      0.98      0.94       298
           3       0.98      0.97      0.97       344
           4       1.00      0.99      0.99       358

    accuracy                           0.96      1665
   macro avg       0.96      0.96      0.96      1665
weighted avg       0.96      0.96      0.96      1665



In [31]:
efficientnet_model.save("UBCeffnetv3.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
from sklearn.metrics import balanced_accuracy_score
print(balanced_accuracy_score(ytest, pred))

0.9618911596330934


In [32]:
import pickle
with open('label_encoder.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

In [7]:
!pip install albumentations

^C
ERROR: Operation cancelled by user
